# Neural Networks - Graph view

After this lecture you should:
* understand why we need non-linearities
* get to know the most common activation functions
* understand the computational graph abstraction and how it relates to the training procedure of neural networks
* know the biggest difference in traditional and deep learning feature representations  [dense vs one-hot representations]

### Recap: Feed-forward Neural Network

We have seen how a neural network can be formalized, both algebraically and graphically. We can think of a feed-forward NN as a function $NN(\mathbf{x})$: $$y= NN(\mathbf{x}) $$

with:
* input: $\mathbf{x}$ (vector with $d_{in}$ dimensions)
* output: $\mathbf{y}$ (output with $d_{out}$ dimensions)

Recall our network: <img src="pics/nn.png"> 
$$NN_{MLP1}(\mathbf{x})=g(\mathbf{xW^1+b^1})\mathbf{W^2}+\mathbf{b^2}$$



Recall that you can also see it as a set of vector-matrix operations:

* In the first layer, the input of 4 dimensions ($x_{1x4}$) is transformed into a vector of 5 dimensions:

$$\textbf{x}_{1x4} \cdot \textbf{W}^1_{4x5} \rightarrow \textbf{v}_{1x5}$$

to which the bias vector is added, and the whole is sent through the activation function to calculate the hidden layer values:

$$\textbf{v}_{1x5} + \textbf{b}_{1x5} \rightarrow \mbox{apply } g \rightarrow \textbf{h}_{1x5}$$

and so forth until the end:


$$\mathbf{h1}=g(\mathbf{xW^1+b^1})$$
$$NN_{MLP1}(\mathbf{x})=\mathbf{h1}\mathbf{W^2}+\mathbf{b^2}$$

Which sizes do $\mathbf{W^2}$ and $\mathbf{b^2}$ have?

answer: W2 = 5x1 b2 = 1x1

What we just did (calculate the output from the input) is also called the **forward pass** in a neural network.


In [5]:
import numpy as np
# forward-pass of a simple neural network:
f = lambda x: 1.0/(1.0 + np.exp(-x)) # activation function (here we use sigmoid)
x = np.random.randn(4) # random input vector of four numbers (1x4) 
print(x)
W1 = np.zeros((4,5))   # Weights W1 (4x5). Note that the weights are commonly initialised randomly, e.g. with Numpy's randn
W2 = np.zeros((5,1))   # Weights W2 (5x1)
b1 = np.zeros((1,5))
b2 = np.zeros((1,1))
h1=f(np.dot(x,W1)+b1) # calculate the activations of the first hidden layer (1x5) - linear transformation followed by non-linearity!
out=f(np.dot(h1,W2)+b2) # calculate output (1x1)
print(out)
print("the network has no weights yet..")

[-0.43502835  1.16713335 -2.00689194 -1.61879718]
[[0.5]]
the network has no weights yet..


All $\mathbf{W}$ and $\mathbf{b}$ are the **parameters** (or **weights**) of the model. They are commonly referred to as θ.

### Updating the weights

### Intuition
We want to adjust the weights so that *a small change in the input should have a small effect in the output*.


In case of a simple model, such as a simple perceptron, a small change might often have a large effect. Remember, the decision function for the perceptron is a threshold, this can be seen as a **step function**: <img src="https://upload.wikimedia.org/wikipedia/commons/a/ac/HardLimitFunction.png" width=300> 

It is 0 for everything below 0 and 1 for positive inputs. If you are already close to the threshold, a small change might have a large effect.
<img src="http://neuralnetworksanddeeplearning.com/images/tikz8.png">

For another reason that we will see later, we will not use simple thresholding, i.e., a **step function**, but rather a smoother function like the **sigmoid** function.

<img src="https://what-if.xkcd.com/imgs/whatif-logo.png">

### What if all the non-linearities in an NN suddenly vanished?

<small>(CREDITS: The following slide has been taken from AJ and ZA's tutorial):</small>


For now, let's simply ignore the bias term to simplify our notation. 

A neural network with an input layer, a middle layer, and an output layer computes the following:

$$\mathbf{y} = g(W^{(2)}g(W^{(1)}g(W^{(0)}\mathbf{x})))$$

$g$ is a non-linearity, which could be different for each layer.

If we change $g$ to a linear function (e.g. a scaling factor), it can simply be multiplied into the weights matrices. Below we assume that $g = 1$, which allows us to simplify the expression:

$$\mathbf{y} = (W^{(2)}(W^{(1)}(W^{(0)}\mathbf{x})))$$

Since matrix multiplication is associative:

$$A(BC) = (AB)C,$$

we can get rid of the brackets altogether:

$$\mathbf{y} = W^{(2)}W^{(1)}W^{(0)}\mathbf{x}.$$

The series of linear transformations can be summarized in a single transformation matrix :

$$T = W^{(2)}W^{(1)}W^{(0)}.$$

And so the prediction of the neural network becomes:

$$\mathbf{y} = T\mathbf{x}.$$

The effective number of parameters in the now non non-linear neural network is $|\mathbf{y}| \times |\mathbf{x}|$, which is precisely the same as a standard linear model.

i.e., **the non-linearities are crucial**!

### Commonly-used activation functions
Tanh: <img src="http://cs231n.github.io/assets/nn1/tanh.jpeg">
Sigmoid: <img src="http://cs231n.github.io/assets/nn1/sigmoid.jpeg">
ReLu: <img src="http://cs231n.github.io/assets/nn1/relu.jpeg">


### So, where do the weights come from?

It's an **optimization** problem. We want to find the weights that "work best". 

<img src="pics/mountains_at_home.jpg">

### Training a Neural Network: Ingredients

* we need to **define what "works best" means**
* we need **a way to change the model (parameters, θ)** to get closer to a good model


### Defining what works best ~ how close we are: Loss

The loss measures how 'far' we are from true solution:

$$L(\mathbf{\hat{y}},\mathbf{y})$$

For multi-class classification the **cross-entropy** is a commonly used loss function: 

$$L_{crossentropy}(\mathbf{y},\mathbf{\hat{y}})= - \sum_i y_i log(\hat{y}_i)$$

For regression the **mean squared error (MSE)** is a common loss function:
$$L_{MSE}(\mathbf{y},\mathbf{\hat{y}}) = \frac{1}{n}\sum_{i=1}^{n}({y-\hat{y}})^2$$


### How to get closer to a good model


**Strategy 1:** random guessing

**Strategy 2:** start with some random initial parameters (weights), and randomly adjust them

**Strategy 3:** follow the gradient: analytical method to find the best direction along which we should change our weight vector: **gradient descent**

<img src="https://upload.wikimedia.org/wikipedia/commons/6/6d/Error_surface_of_a_linear_neuron_with_two_input_weights.png">

### To sum up: Ingredients for training a Neural Network

* we need to define what "works best" means 
    $\rightarrow$ minimize some **loss**
* we need a way to update the parameters to get closer to a good model
    $\rightarrow$ **minimize loss using a gradient-based method: gradient descent**



Intuitively, training a neural networks involves the following steps:

* compute the gradient of the loss function with respect to the parameters
* move the parameters in the negative direction of the gradient to decrease the loss

#### Skeleton of gradient descent:
    
**Input**: training set, loss function $L$

Repeat for number of iterations (**epochs**): 
 
* do a forward pass
* compute the loss on the data: $L(X,Y)$
* compute the gradients: $\mathbf{g} = L(X,Y)$ with respect to $w$ (backpropagation)
* update the parameters, "moving" them in the negative direction of the gradient: $w = w - \eta \mathbf{g}$

## Computational Graph

We want to get an intuitive understanding of the **backpropagation** algorithm. Backpropagation is a way of computing **gradients** of expressions by applying the **chain rule**. But before we get into details of gradients etc, let's introduce the **computational graph abstraction**.


We can represent our neural network as a **computational graph**: nodes are operations, gray boxes are parameters.

<img src="pics/yg-compgraph1.png">

This corresponds to the neural network we have seen before (except for some dimensions, e.g. for input it is 150 instead of 4):

$$NN_{MLP1}(\mathbf{x})=g(\mathbf{xW^1+b^1})\mathbf{W^2}+\mathbf{b^2}$$

<img src="pics/nn.png" width=300> 

#### Why a computational graph?

It helps us to understand the flow of parameters in the model. 

What do we need to compute? The gradient of the loss function with respect to the parameters.

**What's a gradient?** A vector of partial derivatives. 


Recall: the **derivative** 

A derivative gives us a linear approximation of the function at a specific point. Intuitively, the derivative indicates the rate of change of a function $f$ with respect to a variable $x$ (surrounding the region around point $h$):

    
<img src="http://www.intuitive-calculus.com/images/what-is-a-derivative-4.gif">

### Gradient

We are interested in finding the gradient, i.e., all partial derivatives, since our functions are not just functions of single parameters, but of a lot of parameters. 


### Example: gradient

Let's take a simple example of a function: $$f(x) = (x * y)$$ (or simply): $$f(x)=xy$$

We want to calculate the gradient, the vector of partial derivatives (how much does the function change wrt the parameters x and y): $$\nabla f = [\frac{\partial f}{\partial x}, \frac{\partial f}{\partial y}]$$


The partial derivatives of this function are:

$$f(x,y) = x y \hspace{0.5in} \rightarrow \hspace{0.5in} \frac{\partial f}{\partial x} = y \hspace{0.5in} \frac{\partial f}{\partial y} = x$$

In [7]:
## f(x) = x * y
# let's take some numbers 
x = 4
y = -3

In [8]:
## forward pass (function application)
f = x * y
print(f)

-12


In [10]:
## the derivative wrt each variable tells us the sensitivity of the whole expression on its value. 
## for instance, take the partial derivative of f wrt y:
df_dy = x  # it's simply x 
print(x) # this means if we increase the y by a tiny amount, the whole function would increase by this amount.

4


In [11]:
## similarly, the partial derivative of f wrt x is:
df_dx = y
print(y)  # changing x by some small amount would make the whole expression decrease (negative sign)

-3


### Example 2: computational graph - compound expression

(Thanks to lecture notes by Fei-Fei, Karphaty and Johnson, cf: http://cs231n.github.io/optimization-2/)

Let's take the function: $$f(x,y,z) = (x + y) * z$$ 

Let's represent it as a computational graph (green: forward pass values):
<img src="pics/k1.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k2.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k3.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k4.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k5.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k6.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k7.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k8.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k9.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k10.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k11.png"> Slide by Fei-Fei, Karpathy and Johnson

<img src="pics/k13.png"> Slide by Fei-Fei, Karpathy and Johnson

#### Using the chain rule

* we can compute the gradients of the loss along the backward path in our computational graph
* once we know the gradients: we know how much we should change our parameters (in negative direction of gradients, as we want to minimize the loss): $w \pm -\eta \mathbf{g}$





###### Training procedure

Repeat for number of iterations (**batches**): 
* forward pass
* compute loss on data: $L(y,\hat{y})$
* compute gradients: $\mathbf{g} = L(y,\hat{y})$ with respect to $w$
* move parameters in opposite direction of gradient: $w \pm -\eta \mathbf{g}$


#### In practice:

* **gradient descent algorithm** (SGD, Adam, etc.)
* **mini-batches** (use a small subset of training instances) (minibatch/batch size)
* **further hyperparameters**: learning rate $\eta$ (how big a step we take), number of epochs (how many times we go over the training data)

## Input representations in neural networks / deep neural networks

In deep learning we usually work with **dense** representations. Each feature is a vector of numbers.

The computational graph from above is actually not yet complete, it has no input and no loss! 

Hence, the first step is to extend it to define the input. In this case we **embed** the features (each feature gets a n-dimensional vector) and use that as input (here we use the concatenation of 3 word vectors):
<img src="pics/yg-compgraph2.png">

**a) sparse representation vs b) dense representation**

<img src="pics/sparsevsdense.png"> (Illustration from Goldberg's primer)

The computational graph is still not complete, it misses the loss that is observed at the end of the forward pass, and used to backpropagate the error signal. Now we have a complete computational graph: <img src="pics/yg-compgraph3.png">

## References

* Yoav Goldberg's primer chapter 6: [A Primer on Neural Network Models for Natural Language Processing](http://arxiv.org/abs/1510.00726)
* Fei-Fei, Karpathy and Johnson's lecture notes: http://cs231n.github.io/optimization-2/